In [1]:
# first neural network with keras tutorial
import pandas as pd
import pandas_ta as ta
import random
from collections import deque
import numpy as np
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Flatten, Activation, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint, Callback, TensorBoard, LearningRateScheduler
import time
import statistics
from sklearn import preprocessing
from matplotlib import pyplot as plt
from IPython.display import clear_output
import os.path
from numpy import loadtxt
import yfinance as yf
import math
pd.options.mode.chained_assignment = None  # default='warn'

In [6]:
## getting data ##
def getDF(ticker):
    # Load data
    df = pd.read_csv('BTC-USD_1hour.csv')
    df = df.drop(columns=['Unnamed: 0'])
    df = df.tail(900)
    
    # add data points
    df['open_per1'] = df.ta.percent_return(length=1, close = df['open'])*100
    df['ema10'] = df.ta.ema(length=10, close = df['open'])
    df['ema3'] = df.ta.ema(length=3, close = df['open'])
    df['rsi7'] = df.ta.rsi(length=7, close = df['open'])/100
    df['rsi14'] = df.ta.rsi(length=14, close = df['open'])/100
    
#     df = df[[
#             'open','high','low','rsi14','close','open_per1','ema10','O_C_percent'
#             ]]

    del df['timestamp']
    del df['trades']
    del df['volume']
    
    df = df.dropna()
    df.reset_index(inplace=True)
    
    return df

In [7]:
ticker = "TNA"
df = getDF(ticker)
df.tail(5)

,index,open,high,low,close,open_per1,ema10,ema3,rsi7,rsi14
881,1996,39466.30,39468.30,39337.11,39407.02,0.239128,39360.752212,39439.147056,0.614645,0.469731
882,1997,39407.02,39611.31,39361.85,39602.27,-0.150204,39369.164537,39423.083528,0.551462,0.451139
883,1998,39602.27,39686.74,39441.29,39458.78,0.495470,39411.547349,39512.676764,0.678469,0.518709
884,1999,39458.78,39533.00,39383.49,39529.36,-0.362328,39420.135103,39485.728382,0.545930,0.472657
885,2000,39529.36,39569.60,39284.67,39373.74,0.178870,39439.994176,39507.544191,0.591702,0.496343


In [9]:
df['close']

0      36941.07
1      36794.13
2      36660.46
3      36508.00
4      36634.00
         ...   
881    39407.02
882    39602.27
883    39458.78
884    39529.36
885    39373.74
Name: close, Length: 886, dtype: float64

In [10]:
close_6 = list(df['close'].shift(6))

In [11]:
df['close_6'] = close_6

In [12]:
df.head(10)

,index,open,high,low,close,open_per1,ema10,ema3,rsi7,rsi14,close_6
0,1115,37231.37,37517.37,36941.07,36941.07,-0.040299,36922.566312,37218.286454,0.634207,0.612470,NaN
1,1116,36941.07,37125.71,36790.09,36794.13,-0.779719,36925.930619,37079.678227,0.510844,0.537427,NaN
2,1117,36794.13,36889.69,36446.28,36660.46,-0.397769,36901.966870,36936.904114,0.458211,0.503780,NaN
3,1118,36660.46,36861.48,36436.35,36508.00,-0.363292,36858.056530,36798.682057,0.413045,0.474666,NaN
4,1119,36508.00,36692.15,36339.43,36634.00,-0.415870,36794.409888,36653.341028,0.365151,0.443206,NaN
5,1120,36634.00,36665.37,36348.54,36457.29,0.345130,36765.244454,36643.670514,0.428990,0.474221,NaN
6,1121,36457.29,36586.88,36176.78,36282.54,-0.482366,36709.252735,36550.480257,0.368379,0.437420,36941.07
7,1122,36282.54,36989.81,36230.35,36928.84,-0.479328,36631.668601,36416.510129,0.316747,0.404029,36794.13
8,1123,36928.84,37101.22,36631.94,36777.47,1.781298,36685.699765,36672.675064,0.574236,0.542982,36660.46
9,1124,36777.47,37342.47,36701.78,37210.86,-0.409896,36702.385262,36725.072532,0.520625,0.512824,36508.00


In [13]:
df['O_C_percent'] = list(100*(df['close_6']-df['open'])/df['open'])

value = df['O_C_percent'].std()*0.5
print("std: {}".format(round(float(value),2)))

std: 0.84
